In [35]:
import os
import re
import numpy as np
import pandas as pd


In [36]:
def get_file_list(path, file_extn):
    pdf_list = []
    for fn in os.listdir(path):
        match = re.findall("\s*\d+\\."+file_extn, fn)
        if match:
            pdf_list.append(os.path.join(path,fn))
    if len(pdf_list) == 0:
        print ("Could not find any \s files in the directory:".format(file_extn), path)
    return pdf_list



In [37]:

def wrangle_results(results_fn):
    # Prepare the 2016 Results file
    df = pd.read_csv(results_fn)
    df.drop(df.columns[0], axis=1, inplace=True)

    newcols1 = ['Constituency', 'District', 'Candidate', 'Party', 'Votes']
    part1 = df.iloc[:,0:5]
    part1.columns = newcols1
    newcols2 = newcols1[2:5]
    part2 = df.iloc[:,5:8]
    part2.columns = newcols2
    part3 = df.iloc[:,8:11]
    part3.columns = newcols2

    comp = pd.concat([part1, part2, part3], axis=0)
    comp.reset_index(inplace=True)
    comp.drop('index', axis=1, inplace=True)
    comp['Alliance'] = pd.DataFrame(['UDF']*140 + ['LDF']*140 + ['NDA']*140)
    comp['Constituency'] = pd.concat([comp.iloc[0:140,1]] * 3).tolist()
    comp['District'] = pd.concat([comp.iloc[0:140,2]] * 3).tolist()
    comp['Votes'] = comp['Votes'].apply(pd.to_numeric, errors = 'coerce')

    return comp


In [38]:
resultsFile = "lacResults2016.csv"
const_results = wrangle_results(resultsFile)

In [39]:
filesPath = 'ge2016/csvs/'
csvs = get_file_list(filesPath, 'csv')

In [40]:

def get_boothwise_results(fileList, const_results):
    finalFrame = pd.DataFrame()
    i = 1
    for fn in fileList:
        # Now go through each constituency file and look up the candidate names from the results file
        const1 = pd.read_csv(fn, skiprows=[0,1])
        total_line_idx = const1[const1[const1.columns[0]] == "Total Votes Polled"].index[0]
        last_idx = const1.shape[0]
        const1.drop(const1.index[range(total_line_idx + 1,last_idx)], axis=0, inplace=True)

        const1[const1.columns[1:22]] = const1[const1.columns[1:22]].apply(pd.to_numeric, errors='coerce')
        c1 = const1.transpose()
        c1.columns = c1.iloc[0,:]
        c1.drop(c1.index[0], axis=0, inplace=True)
        c1.columns.name=None
        c1 = c1.apply(pd.to_numeric, errors = 'coerce')

        results = pd.merge(c1, const_results, left_on="Total Votes Polled", right_on='Votes', how='inner', suffixes=('_x', '')).dropna(axis=1)
        if results.shape[0] == 0:
            print(i, results)
        i+=1
        finalFrame = pd.concat([finalFrame, results], axis = 0)
        finalFrame.reset_index(inplace=True)
        finalFrame.drop(['index'], axis=1, inplace=True)

    return finalFrame



In [47]:
booth_results = get_boothwise_results(csvs, const_results)

In [48]:
booth_results.head()

,001,001A,002,002A,003,004,004A,005,006,007,...,196,Alliance,Candidate,Constituency,District,No.of Votes recorded on postal ballot papers(To be filled in the case of election from assembly constituency),Party,Total No of Votes recorded at Polling,Total Votes Polled,Votes
0,383.0,NaN,303.0,NaN,356.0,403.0,NaN,250.0,412.0,697.0,...,NaN,UDF,P.B. Abdul Razak,Manjeshwar,Kasaragod,42.0,IUML,56828.0,56870.0,56870.0
1,233.0,NaN,170.0,NaN,124.0,103.0,NaN,102.0,248.0,162.0,...,NaN,LDF,C.H. Kunjambu,Manjeshwar,Kasaragod,68.0,CPI(M),42497.0,42565.0,42565.0
2,367.0,NaN,412.0,NaN,402.0,447.0,NaN,408.0,533.0,140.0,...,NaN,NDA,K. Surendran (politician),Manjeshwar,Kasaragod,190.0,BJP,56591.0,56781.0,56781.0
3,615.0,NaN,590.0,NaN,709.0,406.0,NaN,502.0,491.0,637.0,...,NaN,UDF,N.A. Nellikkunnu,Kasargod,Kasaragod,104.0,IUML,64623.0,64727.0,64727.0
4,259.0,NaN,145.0,NaN,73.0,191.0,NaN,235.0,419.0,352.0,...,NaN,NDA,K. Ravisha Tantri,Kasargod,Kasaragod,290.0,BJP,55830.0,56120.0,56120.0


In [49]:
booth_results.drop(booth_results.columns[[-1, -2, -3, -5]], axis=1, inplace=True)
booth_results.head()

,001,001A,002,002A,003,004,004A,005,006,007,...,192,193,194,195,196,Alliance,Candidate,Constituency,District,Party
0,383.0,NaN,303.0,NaN,356.0,403.0,NaN,250.0,412.0,697.0,...,NaN,NaN,NaN,NaN,NaN,UDF,P.B. Abdul Razak,Manjeshwar,Kasaragod,IUML
1,233.0,NaN,170.0,NaN,124.0,103.0,NaN,102.0,248.0,162.0,...,NaN,NaN,NaN,NaN,NaN,LDF,C.H. Kunjambu,Manjeshwar,Kasaragod,CPI(M)
2,367.0,NaN,412.0,NaN,402.0,447.0,NaN,408.0,533.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NDA,K. Surendran (politician),Manjeshwar,Kasaragod,BJP
3,615.0,NaN,590.0,NaN,709.0,406.0,NaN,502.0,491.0,637.0,...,NaN,NaN,NaN,NaN,NaN,UDF,N.A. Nellikkunnu,Kasargod,Kasaragod,IUML
4,259.0,NaN,145.0,NaN,73.0,191.0,NaN,235.0,419.0,352.0,...,NaN,NaN,NaN,NaN,NaN,NDA,K. Ravisha Tantri,Kasargod,Kasaragod,BJP


In [75]:
constWise = pd.melt(booth_results, id_vars=['Constituency', 'Alliance', 'Candidate', 'District', 'Party'], var_name='Polling Station', value_name='Votes')
constWise.head()

,Constituency,Alliance,Candidate,District,Party,Polling Station,Votes
0,Manjeshwar,UDF,P.B. Abdul Razak,Kasaragod,IUML,001,383.0
1,Manjeshwar,LDF,C.H. Kunjambu,Kasaragod,CPI(M),001,233.0
2,Manjeshwar,NDA,K. Surendran (politician),Kasaragod,BJP,001,367.0
3,Kasargod,UDF,N.A. Nellikkunnu,Kasaragod,IUML,001,615.0
4,Kasargod,NDA,K. Ravisha Tantri,Kasaragod,BJP,001,259.0


In [76]:
constNums = pd.DataFrame({"Constituency": boothwise_results.Constituency.unique()})
constNums['Constituency Num'] = constNums.index.to_series().apply(pd.to_numeric) + 1
constNums.iloc[63:74,1] = constNums.iloc[63:74,1] + 1
constNums.iloc[74:,1] = constNums.iloc[74:,1] + 2
constNums.iloc[60:80,:]

,Constituency,Constituency Num
60,Chelakkara (SC),61
61,Kunnamkulam,62
62,Guruvayoor,63
63,Wadakkanchery,65
64,Ollur,66
65,Thrissur,67
66,Nattika (SC),68
67,Kaipamangalam,69
68,Irinjalakuda,70
69,Puthukkad,71


In [77]:
finalForm = pd.merge(constWise, constNums, on='Constituency', how='inner', suffixes=['', ''])

In [78]:
finalForm.head()

,Constituency,Alliance,Candidate,District,Party,Polling Station,Votes,Constituency Num
0,Manjeshwar,UDF,P.B. Abdul Razak,Kasaragod,IUML,001,383.0,1
1,Manjeshwar,LDF,C.H. Kunjambu,Kasaragod,CPI(M),001,233.0,1
2,Manjeshwar,NDA,K. Surendran (politician),Kasaragod,BJP,001,367.0,1
3,Manjeshwar,UDF,P.B. Abdul Razak,Kasaragod,IUML,001A,NaN,1
4,Manjeshwar,LDF,C.H. Kunjambu,Kasaragod,CPI(M),001A,NaN,1


In [79]:
finalForm.dropna(axis=0, how='any', inplace=True)
finalForm.shape

(63082, 8)

In [80]:
finalForm = finalForm[finalForm.columns[[7,0,3,2,4,1,5,6]]]

In [83]:
finalForm.to_csv('resultsByConstPollStation.csv')